In [1]:
import pandas as pd
import numpy as np

In [2]:
class Graph:
    
    def __init__(self, nodes):
        self.graph = {}
        self.all_nodes = nodes
        
        for node in nodes:
            self.graph[node]=[]
    
    def add_edge (self, start_node, end_node):
        self.graph[start_node].append(end_node)
    
    def make_Q_matrix (self):
        Q = {}
        for node_i in self.all_nodes:
            Q[node_i] = {}
            for node_j in self.all_nodes:
                Q[node_i][node_j] = 0
        
        for node in self.all_nodes:
            for item in self.graph[node]:
                Q[node][item] += (1/(len(self.graph[node]))) #THIS WAS MY MISTAKE
        return(Q)
    

In [3]:
df = pd.read_csv('sklearn-anatomy.csv')

In [4]:
some_nodes = np.asarray(df['page_url'])
some_nodes = np.unique(some_nodes)

other_nodes = np.asarray(df['link_url'])
other_nodes = np.unique(other_nodes)

all_nodes  = np.hstack((some_nodes, other_nodes))
all_nodes = np.unique(all_nodes)

print(len(all_nodes))

6026


In [5]:
graph = Graph(all_nodes)
for index, row in df.iterrows():
    graph.add_edge(row['page_url'], row['link_url'])


In [6]:
tabale = graph.make_Q_matrix()
Q = pd.DataFrame(tabale)
Q = np.asarray(Q)

In [8]:
Q = np.array([[0, 1/3, 0, 1/6, 0, 0], 
              [1/3, 0, 0, 1/6, 0, 0], 
              [0, 1/3, 0, 1/6, 1/2, 1], 
              [1/3, 0, 0, 1/6, 0, 0], 
              [1/3, 1/3, 0, 1/6, 0, 0], 
              [0, 0, 1, 1/6, 1/2, 0], 
              ])

In [9]:
n,_ = Q.shape

P = np.zeros((n, n))
for i in range(n):
    if sum (Q[:,i]) == 0 :
        for j in range (n):
            P[j][i] = 1/n
    else:
        for j in range (n):
            P[j][i] = Q[j][i]     

A = 0.6*P + 0.4*(1/n)*np.ones((n, n))
# A = np.copy(P)

In [10]:
r0 = np.ones((n, 1)) * (1/n)
q = np.dot(A, r0)
r = q * (1/np.linalg.norm(q , ord=1))

itrr = 1
while (np.linalg.norm( (r-r0) , ord=1) > 1.00000000e-16): 
    itrr += 1
    r0 = r
    q = np.dot(A, r)
    r = q * (1/np.linalg.norm(q , ord=1))


print("iteration that we did for algorithm: ", itrr)
print("output of algorithm: ")
print(r)
print(len(r))

iteration that we did for algorithm:  66
output of algorithm: 
[[0.0952381 ]
 [0.0952381 ]
 [0.30595238]
 [0.0952381 ]
 [0.11428571]
 [0.29404762]]
6


In [9]:
dict_scores = {}
r_index = 0
for node in all_nodes:
    dict_scores[node] = r[r_index][0]
    r_index += 1

In [10]:
maxx = np.argsort(r,axis=None)[-1]
minn = np.argsort(r,axis=None)[0]

first_rank = all_nodes[maxx]
score_of_first_rank = r[maxx][0]
last_rank = all_nodes[minn]
score_of_last_rank = r[minn][0]

print(first_rank, 'has the most rank with', score_of_first_rank, 'score')
print(last_rank, 'has the lowest rank with', score_of_last_rank, 'score')

https://scikit-learn.org/stable/glossary.html has the most rank with 0.0068325075352989335 score
https://scikit-learn.org/stable/index.html has the lowest rank with 0.00014355022367849594 score


In [11]:
print(dict_scores['https://scikit-learn.org/stable/glossary.html'])
print(dict_scores['https://scikit-learn.org/stable/index.html'])

0.0068325075352989335
0.00014355022367849594
